In [1]:
import os
import numpy as np
import librosa.display

In [6]:
n_row = 30
n_col = 431
features = np.zeros([1, n_row*n_col])
features.shape

(1, 12930)

In [7]:
for file in os.listdir('.'):
    if file.endswith(".wav"):
        y, sr = librosa.load(file)
        mfccs = librosa.feature.mfcc(y, sr=sr, n_mfcc = n_row)
        print("---------------- Filename and its feature size:", file, mfccs.shape)
        
        if len(mfccs[0])>n_col:
            mfccs = mfccs[:, 0:n_col]
            print("Shorten feature size to:", mfccs.shape)
            
        if len(mfccs[0])<n_col:
            t=np.zeros([n_row, n_col-len(mfccs[0])])
            mfccs = np.hstack((mfccs, t))
            print("Lengthen feature size to:", mfccs.shape)
            
        features = np.vstack((features, mfccs.reshape(1,-1)))

---------------- Filename and its feature size: c001.wav (30, 431)
---------------- Filename and its feature size: c002.wav (30, 431)
---------------- Filename and its feature size: c003.wav (30, 432)
Shorten feature size to: (30, 431)
---------------- Filename and its feature size: c004.wav (30, 431)
---------------- Filename and its feature size: c005.wav (30, 431)
---------------- Filename and its feature size: c006.wav (30, 433)
Shorten feature size to: (30, 431)
---------------- Filename and its feature size: c007.wav (30, 431)
---------------- Filename and its feature size: c008.wav (30, 431)
---------------- Filename and its feature size: c009.wav (30, 431)
---------------- Filename and its feature size: c010.wav (30, 431)
---------------- Filename and its feature size: c011.wav (30, 431)
---------------- Filename and its feature size: c012.wav (30, 432)
Shorten feature size to: (30, 431)
---------------- Filename and its feature size: c013.wav (30, 431)
---------------- Filenam

---------------- Filename and its feature size: v011.wav (30, 431)
---------------- Filename and its feature size: v012.wav (30, 431)
---------------- Filename and its feature size: v013.wav (30, 431)
---------------- Filename and its feature size: v014.wav (30, 431)
---------------- Filename and its feature size: v015.wav (30, 431)
---------------- Filename and its feature size: v016.wav (30, 430)
Lengthen feature size to: (30, 431)
---------------- Filename and its feature size: v017.wav (30, 431)
---------------- Filename and its feature size: v018.wav (30, 434)
Shorten feature size to: (30, 431)
---------------- Filename and its feature size: v019.wav (30, 432)
Shorten feature size to: (30, 431)
---------------- Filename and its feature size: v020.wav (30, 431)
---------------- Filename and its feature size: v021.wav (30, 432)
Shorten feature size to: (30, 431)
---------------- Filename and its feature size: v022.wav (30, 431)
---------------- Filename and its feature size: v023.wa

In [8]:
#Remove the first line
features = features[1:, :]
#Save the feature set to a file
np.save("30features.npy", features)

In [9]:
#features=np.load("features.npy")
features.shape

(200, 12930)

In [10]:
labels=np.hstack((np.zeros((100,),dtype=int),np.ones((100,),dtype=int)))

In [11]:
print('Classes and labels:', np.unique(labels), labels)

Classes and labels: [0 1] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [82]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels , test_size=.2, random_state=11)

In [83]:
#Create the SVM model
from sklearn.svm import SVC
clf = SVC(kernel = 'rbf', C=10**2, random_state = 1)

#Fit the model for the data
clf.fit(X_train, y_train)

#Make the prediction
y_pred = clf.predict(X_test)

In [86]:
from sklearn import metrics

# Model Accuracy: 
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
# Model Precision: 
print("Precision: {:.2f}".format(metrics.precision_score(y_test, y_pred)))
# Model Recall: 
print("Recall: {:.2f}".format(metrics.recall_score(y_test, y_pred)))

Accuracy: 0.775
Precision: 0.63
Recall: 0.86


In [85]:
print(metrics.confusion_matrix(y_test,y_pred))
print(metrics.classification_report(y_test,y_pred))

[[19  7]
 [ 2 12]]
              precision    recall  f1-score   support

           0       0.90      0.73      0.81        26
           1       0.63      0.86      0.73        14

    accuracy                           0.78        40
   macro avg       0.77      0.79      0.77        40
weighted avg       0.81      0.78      0.78        40



In [80]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = clf, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
#print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 77.50 %


In [81]:
clf_SVC = SVC(C=100.0, kernel='rbf', degree=3, gamma='auto', coef0=0.0, shrinking=True, 
          probability=False, tol=0.001, cache_size=200, class_weight=None, 
          verbose=0, max_iter=-1, decision_function_shape="ovr", random_state = 0)

clf_SVC.fit(X_train,y_train)
#Make the prediction
y_pred = clf_SVC.predict(X_test)

# Model Accuracy: 
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
# Model Precision: 
print("Precision:", metrics.precision_score(y_test, y_pred))
# Model Recall: 
print("Recall:", metrics.recall_score(y_test, y_pred))

print('Accuracy of SVC on training set: {:.2f}'.format(clf_SVC.score(X_train, y_train) * 100))

print('Accuracy of SVC on test set: {:.2f}'.format(clf_SVC.score(X_test, y_test) * 100))

Accuracy: 0.5
Precision: 0.5
Recall: 1.0
Accuracy of SVC on training set: 100.00
Accuracy of SVC on test set: 50.00
